<font size='5'>Mixture of Poisson Distributed Data</font>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import arviz as az 

import pymc3 as pm

In [ ]:
data1=np.random.lognormal(mean=0.,sigma=1., size=100)
data2=np.random.lognormal(mean=0.,sigma=2., size=50)
data=np.concatenate(([ int(x) for x in data1 ], [ int(x) for x in data2 ]))
print(data)

num_samples = 5000

with pm.Model() as model:
    lam1 = pm.Exponential('lam1', lam=1)
    lam2 = pm.Exponential('lam2', lam=1)
    pois1 = pm.Poisson.dist(mu=lam1)
    pois2 = pm.Poisson.dist(mu=lam2)
    w = pm.Dirichlet('w', a=np.array([1, 1]))
    like = pm.Mixture('like', w=w, comp_dists=[pois1, pois2], observed=data)

print(like)
with model:
    trace = pm.sample(num_samples, n_init=10000, tune=10000)[1000:]
#with model:
#    ppc_trace = pm.sample_posterior_predictive(trace, num_samples)
    
az.plot_trace(trace, var_names=["p", "sd", "means"]);
'''
fig, ax = plt.subplots(figsize=(8, 6))
ax.hist(data, bins=30,
        histtype='step', lw=2,
        label='Observed data')
ax.hist(ppc_trace['like'], bins=30, 
        histtype='step', lw=2,
        label='Posterior predictive distribution')
ax.legend(loc=1)
plt.show()
'''